In [1]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

In [2]:
import torch
from io import BytesIO
from PIL import Image
%pylab inline 
import cv2
from IPython.display import clear_output
from sort import *
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Using cache found in /home/cmt/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-1 Python-3.10.9 torch-1.12.0+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
Tensor = torch.cuda.FloatTensor

In [4]:
img_size=416
conf_thres=0.8
nms_thres=0.4
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0).to(device)
    #input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(image_tensor)
        #detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections



In [5]:
videopath = 'etc/vidroom2.mp4'



cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# initialize Sort object and video capture
vid = cv2.VideoCapture(videopath)
mot_tracker = Sort() 

# Save output to avi video
fourcc = cv2.VideoWriter_fourcc(*"XVID")
fps = 25.0
width = 1080
height = 1920
out = cv2.VideoWriter("output.avi", fourcc, fps, (width, height))

#while(True):
counter = 0
while vid.isOpened():
    print("Frame ", counter)
    ret, frame = vid.read()
    if frame is None:
        print('Finished!')
        out.release()
        break
    counter += 1
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.flip(frame, 0)
    pilimg = Image.fromarray(frame)
    detections = model(frame)
    #print("N det: ", len(detections.pandas().xyxy[0]))

    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    #detections.render()  # updates results.ims with boxes and labels
    #for im in detections.ims:
    #    buffered = BytesIO()
    #    im_base64 = Image.fromarray(im)
    #frame = cv2.cvtColor(numpy.array(im_base64), cv2.COLOR_RGB2BGR)
    if detections is not None:
        print("N det: ", len(detections.pandas().xyxy[0]))
        tracked_objects = mot_tracker.update(detections.pandas().xyxy[0].iloc[:,:5].to_numpy())
        print("N tracked: ", len(tracked_objects))

        for x1, y1, x2, y2, obj_id in tracked_objects:
            #x1, y1, x2, y2, obj_id = float(x1), float(y1), float(x2), float(y2), int(obj_id)
            #box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            #box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            #y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            #x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            #cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
            #cv2.rectangle(frame, (x1, y1-35), (x1+len(str(obj_id))*19+60, y1), color, -1)
            #cv2.putText(frame, str(obj_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
            
            # Draw the bounding box around the tracked object
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 3)
            # Save the detections to the video file
            #out.write(frame)
    out.write(frame)
    clear_output(wait=True)


Frame  643
Finished!


In [6]:
detections.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,0.000000,946.034546,566.277893,1918.727417,0.864846,56,chair
1,650.831665,732.959229,878.007202,1035.481445,0.843067,58,potted plant
2,78.568062,645.281128,206.257294,825.170349,0.694664,58,potted plant
3,315.564728,182.156982,427.244690,407.752350,0.662951,39,bottle
4,199.963898,634.428406,304.374695,856.296814,0.656991,58,potted plant
5,271.127106,306.980347,322.899445,414.358429,0.580921,39,bottle
6,391.976349,666.949158,666.850769,987.356018,0.574073,58,potted plant
7,310.032776,183.676102,423.452576,407.236389,0.548258,58,potted plant
8,276.248688,600.474976,533.053894,906.181458,0.516682,58,potted plant


In [7]:
tracked_objects

array([[     269.68,      305.96,      322.98,      415.29,         294],
       [     310.49,      183.46,      422.92,      407.45,         285],
       [     310.55,      182.26,      425.52,      407.93,         272],
       [     648.72,      731.91,      877.77,      1037.2,         260],
       [     402.01,      673.63,      666.34,      988.21,         255],
       [     280.05,      603.18,      524.87,      905.23,         246],
       [     200.02,      636.69,      301.91,      855.23,         240],
       [     78.075,      647.06,      202.96,      825.19,         236],
       [    -2.3038,      941.42,      568.36,      1922.5,         221]])

In [8]:
out.release()